# Using Emsembl-REST

## Import statements

In [8]:
pip install ensembl_rest

Note: you may need to restart the kernel to use updated packages.


In [9]:
import ensembl_rest
import pandas as pd
import requests
import csv
import re
import sys


In [10]:
help(ensembl_rest)

Help on package ensembl_rest:

NAME
    ensembl_rest

PACKAGE CONTENTS
    assemblymapper
    core (package)
    data (package)
    ensemblclient
    utils

FUNCTIONS
    VariantAnnotationSet(*args, **kwargs)
            Variation GA4GH ``POST ga4gh/variantannotationsets/search``
        
        Return a list of annotation sets in GA4GH format
        
        
        **Parameters**
        
        - Required:
        
          + *Name*: 
        
            * *Type*: String
            * *Description*: 
            * *Default*: -
            * *Example Values*: 1
        
        
        - Optional:
        
          + *Name*: callback
        
            * *Type*: String
            * *Description*: Name of the callback subroutine to be returned by the requested JSONP response. Required ONLY when using JSONP as the serialisation method. Please see , .
            * *Default*: -
            * *Example Values*: randomlygeneratedname
        
        
          + *Name*: pageSiz

## Getting data

In [11]:
def read_gene_ids_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            # Skip the first line
            next(file)
            
            # Read gene IDs from the remaining lines
            gene_ids = [line.strip() for line in file]

        return gene_ids
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return []
    except Exception as e:
        print(f"Error: {str(e)}")
        return []


In [15]:
# Function to get cds sequence
def get_cds(transcript_id):
    address = f"https://rest.ensembl.org/sequence/id/{transcript_id}?multiple_sequences=1;type=cds"
    r = requests.get(address, headers={ "Content-Type" : "text/x-fasta"})
    # Ensure that there are no issues with the sequence request (e.g. multiple sequences available but not adequately requested)
    if not r.ok:
        r.raise_for_status()
        sys.exit()
    raw_output = r.text
    # Extract only the nucleotide sequence and format into a single string
    pattern = re.compile('(?:^|\n)[ATGC]+')
    matches = pattern.findall(raw_output)

    return ''.join(matches).replace('\n', '')

def get_promoter_terminator(transcript_id, promoter_length=1000, terminator_length=500):

    sequence = ensembl_rest.sequence_id(id=gene_id, type="cdna", expand_5prime=promoter_length, expand_3prime=terminator_length)["seq"]
    
    # Use expand_5prime and expand_3prime to get promoter and terminator sequences
    promoter_sequence = sequence[:promoter_length]
    terminator_sequence = sequence[-terminator_length:]
    
    return promoter_sequence, terminator_sequence

def extract_utr_information(data):
    utr_data = data.get('UTR', [])
    
    utr5_list = []
    utr3_list = []

    for utr_entry in utr_data:
        utr_type = utr_entry.get('type', '')
        utr_start = utr_entry.get('start', None)
        utr_end = utr_entry.get('end', None)

        if utr_type and utr_start is not None and utr_end is not None:
            if utr_type == 'five_prime_utr':
                utr5_list.append((utr_start, utr_end))
            elif utr_type == 'three_prime_utr':
                utr3_list.append((utr_start, utr_end))

    chromosome = utr_entry['seq_region_name'] if utr_data else None
    strand = data.get('strand', None)

    return utr5_list, utr3_list, chromosome, strand

def get_utr_sequence(chromosome, strand, start, end, species="human"):
    
    return ensembl_rest.sequence_region(region=f"{chromosome}:{start}..{end}:{strand}", species=species)["seq"]

def get_full_utr_sequence(list_utr_coordinates, chromosome, strand):
    
    concatenated_sequence = ""

    for start, end in list_utr_coordinates:
        sequence = get_utr_sequence(chromosome, strand, start, end)
        concatenated_sequence += sequence

    return concatenated_sequence

In [16]:
# Define the gene IDs you are interested in
# gene_ids = ["ENSG00000000003", "ENSG00000000005", "ENSG00000000419"]  # Replace with your gene IDs
file_path = "homo_sapiens_genes_small.txt"
gene_ids = read_gene_ids_from_file(file_path)

# Initialize a CSV writer
csv_file = open("ensembl_data_homo_sapiens.csv", "w", newline="")
csv_writer = csv.writer(csv_file)

# Write the header to the CSV file
csv_writer.writerow(["ensembl_gene_id", "transcript_id", "promoter", "utr5", "cds", "utr3", "terminator"])


# Loop through each gene ID and retrieve the data
for gene_id in gene_ids:
    print(f"Extracting data for gene ID : {gene_id}")
    gene_data = ensembl_rest.lookup(species='homo sapiens', id=gene_id)
    
    # Get transcript ID
    transcript_id = gene_data["canonical_transcript"].split(".")[0]

    # Retrieve promoter, CDS, and terminator sequences
    cds_sequence = get_cds(transcript_id)
    promoter_sequence, terminator_sequence = get_promoter_terminator(transcript_id)
    
    # Retrieve UTR sequences
    transcript_data = ensembl_rest.lookup(id=transcript_id, params={'expand':True,'utr':True})
    utr5_coord_list, utr3_coord_list, chromosome, strand = extract_utr_information(transcript_data)
    utr5_sequence = get_full_utr_sequence(utr5_coord_list, chromosome, strand)
    utr3_sequence = get_full_utr_sequence(utr3_coord_list, chromosome, strand)

    # Write the row to the CSV file
    csv_writer.writerow([gene_id, transcript_id, promoter_sequence, utr5_sequence, cds_sequence, utr3_sequence, terminator_sequence])

# Close the CSV file
csv_file.close()

print("Data extraction complete.")


Extracting data for gene ID : ENSG00000000003
Extracting data for gene ID : ENSG00000000005
Extracting data for gene ID : ENSG00000000419
Extracting data for gene ID : ENSG00000000457
Extracting data for gene ID : ENSG00000000460
Data extraction complete.


In [14]:
genetic_data = pd.read_csv("ensembl_data_homo_sapiens.csv")
genetic_data

,ensembl_gene_id,transcript_id,promoter,utr5,cds,utr3,terminator
0,ENSG00000000003,ENST00000373020,AGCTCTTCAGTAGTTTCTGAACATCTAGACGGTAGGATGTAGAATA...,AGTTGTGGACGCTCGTAAGTTTTCGGCAGTTTCCGGGGAGACTCGG...,ATGGCGTCCCCGTCTCGGAGACTGCAGACTAAACCAGTCATTACTT...,CCCAATGTATCTGTGGGCCTATTCCTCTCTACCTTTAAGGACATTT...,AGGAAACATTTCACCTAATGATAACTGGTACAAAGGAAAGTTCACG...
1,ENSG00000000005,ENST00000373031,AGCCGACTCACTTGCAACTCCACCTCAGCAGTGGTCTCTCAGTCCT...,AGCCGACTCACTTGCAACTCCACCTCAGCAGTGGTCTCTCAGTCCT...,ATGGCAAAGAATCCTCCAGAGAATTGTGAAGACTGTCACATTCTAA...,TAGGAGGTTTGAGCTCAAATGCTTAAACTGCTGGCAACATATAATA...,AGGAGCTTAATTTCTCACAAGCCCCCAACTCATACAGTCTTCTAGT...
2,ENSG00000000419,ENST00000371588,ACTGCGCTAGTGGACAGCCGAGCCCACCGCAGCCCACGATTAGCAC...,AGTTCCGCC,ATGGCCTCCTTGGAAGTCAGTCGTAGTCCTCGCAGGTCTCGGCGGG...,AAGAAAGATACTCATTTATAGTTACGTTCATTTCAGGTTAAACATG...,ATTTTATACTTACTTTAAGGACCTCTGTTGTAGGACTAATCCTATT...
3,ENSG00000000457,ENST00000367771,GCACCTCTACTGTTTGCTACAAGTGGCCAGCAGCCATTTTGGATTT...,GTAGTGGCCACAGCCTTACAGGCAGGCAGGGGTGGTTGGTGTCAAC...,ATGGGATCAGAGAACAGTGCTTTAAAGAGCTATACACTGAGAGAAC...,CAATAGATGTGAGTTAAACTTTAGGAAAAAGGATTCCCTTTTTTTA...,AACTCATGATTCCTTCAGGGTCCCTAAGGATAATACAAAATTAACT...
4,ENSG00000000460,ENST00000359326,AACCCGCTCGGGTCCCCTTCCACACTGTGGAAGCTTTGTTCTTTCG...,ACTGCGAGTTTCCGGTCTGGGCTTTGGCGGGTCTGGTTTGAAGCTC...,ATGTTTTTACCTCATATGAACCACCTGACATTGGAACAGACTTTCT...,AACTTATCACTAGGCAGAACTGGGTTTGATGCTTTGTCAACTGAAA...,TGGATAATGAGCTCCTGGGATGGGAAAAGCAGGCCACTTTGGGGTT...
